# Puzzle 1
[Problem Description](https://adventofcode.com/2025/day/7)

In [1]:
test_input = '''.......S.......
...............
.......^.......
...............
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
...............'''
test_rows = test_input.split('\n')

In [7]:
print(test_rows[1].find('^'))
# -1
print(test_rows[2].find('^'))
# 7
print(test_rows[4].find('^'))
# 6 
# need a helper to find all

-1
7
6


In [ ]:
class TachyonBeam:
    manifold: list[str] #manifold diagram split into rows

    def __init__(self, manifold):
        self.manifold = manifold
    
    def find_all_splitters(self, row_num: int):
        start_ind = 0
        splitter_idxs = []
        row = self.manifold[row_num]
        while start_ind < len(row):
            next_splitter = row.find('^', start_ind)
            if next_splitter == -1:
                return splitter_idxs
            else:
                splitter_idxs.append(next_splitter)
            start_ind = next_splitter +1
    
    def find_start(self):
        return self.manifold[0].find('S')
    
    def find_beam_splits(self, row_num: int, curr_pos: list[int]) -> list[int]:
        """
        Find all the indices where the beam will split in a given row,
        assuming the beam currently occupies a given list of indices.
        """
        splitters = self.find_all_splitters(row_num)
        beam_splits = [idx for idx in splitters if idx in curr_pos]
        return beam_splits
    
    def split_beam(self, row_num: int, curr_pos: list[int]) -> tuple[list[int], int]:
        """
        Split the beam into its new positions on a given row,
        assuming the beam currently occupies a given list of indices.
        Also count the number of splits on this layer.
        """
        beam_splits = self.find_beam_splits(row_num, curr_pos)
        new_pos = curr_pos[:]
        splits = 0
        for idx in beam_splits:
            new_pos.remove(idx) #if we split at idx, the beam is no longer there
            new_pos.extend([idx-1, idx+1]) #but it is in the positions to the left and right
            splits +=1 
        return sorted(list(set(new_pos))), splits
    
    def final_beam(self) -> tuple[list[int], int]:
        """
        Trace the beam through the manifold and return its position when it exits. 
        Puzzle 1's solution is the length of the output. 
        In anticipation of puzzle 2, also track the position of the beam
        at each layer. 
        """
        curr_pos = [self.find_start()]
        splits = 0
        for row_num in range(1, len(self.manifold)):
            curr_pos, new_splits = self.split_beam(row_num, curr_pos)
            splits += new_splits
        return curr_pos, splits
        

In [ ]:
testManifold = TachyonBeam(test_rows)
testManifold.final_beam()
# ([0, 2, 4, 5, 6, 7, 8, 10, 11, 12, 14], 21)
# success

([0, 2, 4, 6, 8, 10, 11, 12, 14], 21)

In [43]:
with open('../data/day7.txt', 'r') as input_fle:
    input_rows = [row.rstrip('\n') for row in input_fle.readlines()]

In [ ]:
manifold = TachyonBeam(input_rows)
print(manifold.final_beam())
# (long list) 1585
# success! 

([0, 2, 4, 5, 7, 9, 10, 12, 13, 14, 16, 18, 19, 20, 22, 24, 25, 26, 28, 30, 31, 32, 34, 36, 38, 40, 42, 43, 45, 46, 47, 49, 50, 52, 54, 55, 57, 58, 60, 62, 64, 66, 67, 68, 69, 71, 72, 74, 76, 78, 80, 82, 84, 85, 86, 88, 90, 91, 92, 94, 96, 97, 98, 99, 101, 102, 104, 105, 107, 108, 110, 112, 114, 116, 118, 120, 122, 124, 126, 128, 129, 130, 132, 134, 135, 136, 138, 140], 1585)


# Puzzle 2
[Puzzle link](https://adventofcode.com/2025/day/7#part2)

Welp, I was totally wrong about the direction of puzzle 2 today!

In [122]:
import itertools
from collections import defaultdict

class QuantumTachyonBeam:
    manifold: list[str] #manifold diagram split into rows

    def __init__(self, manifold):
        self.manifold = manifold

    def find_all_splitters(self, row_num: int):
        start_ind = 0
        splitter_idxs = []
        row = self.manifold[row_num]
        while start_ind < len(row):
            next_splitter = row.find('^', start_ind)
            if next_splitter == -1:
                return splitter_idxs
            else:
                splitter_idxs.append(next_splitter)
            start_ind = next_splitter +1
    
    def find_start(self):
        return self.manifold[0].find('S')
    
    def find_beam_splits(self, row_num: int, curr_pos: list[int]) -> list[int]:
        """
        Find all the indices where the beam will split in a given row,
        assuming the beam currently occupies a given list of indices.
        """
        splitters = self.find_all_splitters(row_num)
        beam_splits = [idx for idx in splitters if idx in curr_pos]
        return beam_splits
    
    def _generate_timelines(self, length):
        return [list(p) for p in itertools.product(['L', 'R'], repeat=length)]
    
    def split_beam(self, curr_timeline: list[int]) -> tuple[list[list[int]], int]:
        """
        Split the beam into all possible new positions on a given row,
        assuming the beam currently occupies a given list of indices.
        Also count the number of splits on this layer.
        """
        curr_pos = curr_timeline[-1]
        beam_splits = self.find_beam_splits(len(curr_timeline), curr_pos)
        new_pos_space = []

        timelines = self._generate_timelines(len(beam_splits)) #all possible choices at this layer

        for timeline in timelines:
            new_timeline = curr_timeline[:]
            new_pos = curr_pos[:]
            for idx in range(len(timeline)):
                split = beam_splits[idx]
                new_pos.remove(split) #if we split at idx, the beam is no longer there
                if timeline[idx] == 'L':
                    new_pos.append(split-1) #but it is in the positions to the left or the right
                else:
                    new_pos.append(split+1)
            new_timeline.append(sorted(list(set(new_pos))))
            new_pos_space.append(new_timeline)
 
        return new_pos_space, len(timelines)  

    def count_timelines(self, curr_timeline: list[int] | None = None) -> int:
        """
        Failed attempt: does not scale to full input size. 
        """
        if curr_timeline is None:
            curr_timeline = [[self.find_start()]]
        total_timelines = 0
        if len(curr_timeline) < len(self.manifold):
            next_timelines, new_num_timelines = self.split_beam(curr_timeline)
            for timeline in next_timelines:
                total_timelines += self.count_timelines(timeline)
        else:
            total_timelines = 1
        return total_timelines
    
    def count_timelines_v2(self) -> int:
        """
        Try 2. Just keep track of how many paths reach the current position
        in each row, and keep a running total of that through the manifold.
        """
        start_col = self.find_start()

        counts = {start_col: 1}
        exited = 0

        for row_idx in range(1, len(self.manifold)):
            row = self.manifold[row_idx]
            new_counts = defaultdict(int)

            for col, ways in counts.items():
                if not (0 <= col < len(row)):
                    exited += ways
                    continue

                tile = row[col]

                if tile == '^':
                    left_col = col - 1
                    if left_col >= 0:
                        new_counts[left_col] += ways
                    else:
                        exited += ways
                    right_col = col + 1
                    if right_col < len(row):
                        new_counts[right_col] += ways
                    else:
                        exited += ways

                else:
                    new_counts[col] += ways
                
            counts = new_counts

        exited += sum(counts.values())
        return exited

In [123]:
quantumTest = QuantumTachyonBeam(test_rows)
quantumTest.count_timelines_v2()
# 40
# success!

40

In [125]:
quantumManifold = QuantumTachyonBeam(input_rows)
quantumManifold.count_timelines_v2()
# 64
# success!

16716444407407

# Reflection

Another day where my part 2 solution was logically correct but computationally infeasible. Sigh. The v1 solution ran out of memory. 

Needed some help on the dynamic programming approach in part 2. This is what I was *trying* to do, but I couldn't quite articulate the "track the n of paths to reach each position" logic. I also didn't notice the caveat in the problem that a beam which exits to the left or right of the manifold still counts as a timeline; I implicitly assumed this wasn't allowed. 

